In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [3]:
# Load the training data
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')

In [4]:
# Feature engineering
def feature_engineering(data):
    data['Title'] = data['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
    data['Cabin'] = data['Cabin'].str[0]
    data['FamilySize'] = data['SibSp'] + data['Parch'] + 1
    data['IsAlone'] = (data['FamilySize'] == 1).astype(int)
    data['AgeBin'] = pd.cut(data['Age'], bins=[0, 12, 20, 40, 60, 80], labels=['Child', 'Teenager', 'Adult', 'MiddleAge', 'Senior'])
    data['FareBin'] = pd.cut(data['Fare'], bins=[0, 7.91, 14.454, 31, 512], labels=['Low', 'Medium', 'High', 'VeryHigh'])
    return data

In [5]:
train_data = feature_engineering(train_data)

In [6]:
# Define numerical and categorical features
numerical_features = ['Age', 'Fare']
categorical_features = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked', 'Title', 'Cabin', 'FamilySize', 'IsAlone', 'AgeBin', 'FareBin']

# Create transformers for numerical and categorical features
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))])

# Combine transformers into a preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)])

In [7]:
# Define the model
model = RandomForestClassifier()

# Create a pipeline
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', model)])

In [8]:
# Split the data
X = train_data.drop(['Survived', 'Name', 'Ticket', 'PassengerId'], axis=1)
y = train_data['Survived']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
# Define a parameter grid for RandomForestClassifier
param_grid = {
    'classifier__n_estimators': [100, 150],
    'classifier__max_depth': [None, 10],
    'classifier__min_samples_split': [2, 5],
    'classifier__min_samples_leaf': [1, 2]
}

In [10]:
# Create a GridSearchCV object
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, n_jobs=-1, scoring='accuracy')

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best parameters and the best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

Best Parameters: {'classifier__max_depth': 10, 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 5, 'classifier__n_estimators': 100}
Best Score: 0.832837584950261


In [11]:
# Evaluate the best model on the validation set
best_model = grid_search.best_estimator_
validation_accuracy = best_model.score(X_valid, y_valid)
print("Validation Set Accuracy:", validation_accuracy)

Validation Set Accuracy: 0.8212290502793296


In [12]:
# Load and preprocess the test data
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
test_data = feature_engineering(test_data)

# Make predictions on the test data
X_test = test_data.drop(['Name', 'Ticket', 'PassengerId'], axis=1)
predictions = best_model.predict(X_test)

In [13]:
# Create a submission DataFrame
submission = pd.DataFrame({
    'PassengerId': test_data['PassengerId'],
    'Survived': predictions
})

# Save the submission to a CSV file
submission.to_csv('submission.csv', index=False)